In [24]:
import pandas as pd
from datetime import datetime
import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



## Function to log progress of the ETL

In [25]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ':' + message + '\n')



## Extraction

In [26]:
def extract(url,table_attribs = None):
    tables = pd.read_html(url, attrs=table_attribs ,match='Bank name')
    df = pd.DataFrame(tables[0][['Bank name', 'Market cap (US$ billion)']])
    
    # Rename the columns
    df.rename(columns={
        'Bank name': 'Name',
        'Market cap (US$ billion)': 'MC_USD_Billion'
    }, inplace=True)
    
    return df

## Transformation

In [27]:
def transform(df, csv_path):
    df1 = pd.read_csv('exchange_rate.csv')
    currencies = df1["Currency"].tolist()
    exchange_rates = df1["Rate"].tolist()
    for currency, rate in zip(currencies, exchange_rates):
        df[f"MC_{currency}_Billion"] = round(df["MC_USD_Billion"] * rate,2)
    return df

## Loading data

In [28]:
def load_data(df, output_path): 
    df.to_csv(output_path) 



def load_to_db(df,sql_connection, table_name):
    df.to_sql(table_name, conn, if_exists='replace', index=False)

## Run Query Function

In [29]:
def run_query(query_statement, sql_connection):
    df = pd.read_sql_query(query_statement, sql_connection)
    print(query_statement)
    print(df)
    print("\n")
    return df

## Running Complete ETL

In [30]:
log_file = "code_log.txt"
target_file = "Largest_banks_data.csv"
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
db_name = 'Banks.db'
table_name ='Largest_banks'
output_path = "Largest_banks_data.csv"
csv_path="exchange_rate.csv"
log_progress("Preliminaries complete. Initiating ETL process")

 
table = extract(url)
print("Data by Market Capitalization \n", table) 
log_progress("Data extraction complete. Initiating Transformation process") 
 

transformed_data = transform(table,csv_path)
print(transformed_data)
print(transformed_data['MC_EUR_Billion'][4])
log_progress("Data transformation complete. Initiating Loading process") 
 
load_data(transformed_data,output_path)
log_progress("Data saved to CSV file") 

 
conn = sqlite3.connect(db_name)
log_progress("SQL Connection initiated") 
 
load_to_db(transformed_data,conn,table_name)
log_progress("Data loaded to Database as a table, Executing queries")

# Query 1: Select all records
run_query("SELECT * FROM Largest_banks", conn)

# Query 2: Calculate average
run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", conn)

# Query 3: Get first 5 names
run_query("SELECT Name FROM Largest_banks LIMIT 5", conn)
log_progress("Process Complete")


conn.close()
log_progress("Server Connection closed")

Data by Market Capitalization 
                                       Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81
                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3      